In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import time

In [ ]:
print(f"TensorFlow version : {tf.__version__}")

In [ ]:
physical_devices = tf.config.list_physical_devices()

print("number of physical devices CPUs/GPUs:", len(physical_devices))
print(physical_devices)

In [ ]:
BATCH_SIZE = 256
print("BATCH SIZE", 7*7*256)
# prepare data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

print("train images", train_images.shape)
print("train labels", train_labels.shape)
print("test images", test_images.shape)
print("test labels", test_labels.shape)
print("---------------------------------------------------------------------------------")
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
print("train images", train_images.shape)

print("---------------------------------------------------------------------------------")
train_images = (train_images - 127.5) / 127.5
print("train images", train_images.shape)

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(60000).batch(256)

print("train_dataset", train_dataset)

In [ ]:
print(train_images.shape)

In [ ]:
# the generator net
def create_generator():
    model = tf.keras.Sequential()
    
    # creating Dense layer with units 7*7*256(batch_size) and input_shape of (100,)
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model
print(create_generator().summary())

In [ ]:
# the discriminator net
def create_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model
print(create_discriminator().summary())

In [ ]:
# loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def D_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def G_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# optimizer
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# hyperparameters
noise_dim = 100
num_of_generated_examples = 16

seed = tf.random.normal([num_of_generated_examples, noise_dim])

generator = create_generator()
discriminator = create_discriminator()

In [ ]:
# the training loop
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = G_loss(fake_output)
        disc_loss = D_loss(real_output, fake_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train_GAN(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for image_batch in dataset:
            train_step(image_batch)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
train_GAN(train_dataset, 500)